In [15]:
import torch
import nibabel as nib
import os
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import numpy as np
import time
from tqdm.auto import tqdm
import utils as utils
from scipy.ndimage import rotate
import matplotlib.pyplot as plt

/home/student/sinaziaee/mini_conda/miniconda3/envs/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle

# Open the .pkl file
with open('final_result14/history_40.pkl', 'rb') as f:
    # Load the object from the file
    obj = pickle.load(f)

# Print the text inside the object
print([f'{i+1}: {100*value:.2f}' for i, value in enumerate(obj['dice_valid_score'])])

['1: 82.05', '2: 78.95', '3: 82.53', '4: 83.47', '5: 83.92', '6: 85.60', '7: 88.58', '8: 90.68', '9: 80.46', '10: 88.54', '11: 90.04', '12: 88.22', '13: 81.30', '14: 92.12', '15: 90.62', '16: 86.84', '17: 83.56', '18: 91.45', '19: 91.88', '20: 89.18', '21: 90.96', '22: 92.71', '23: 89.57', '24: 93.36', '25: 92.09', '26: 94.13', '27: 95.42', '28: 91.54', '29: 94.03', '30: 89.44', '31: 90.32', '32: 93.18', '33: 93.93', '34: 95.31', '35: 95.16', '36: 93.51', '37: 89.65', '38: 93.28', '39: 91.36', '40: 90.90']


In [17]:
def get_current_path(path=None):
    if path:
        return path
    else:
        return os.getcwd()

new_folder = 'data_npy4'

if not os.path.exists(f'{get_current_path()}/{new_folder}'):
    os.mkdir(f'{get_current_path()}/{new_folder}')
    
if not os.path.exists(f'{get_current_path()}/{new_folder}/train'):
    os.mkdir(f'{get_current_path()}/{new_folder}/train')
    
if not os.path.exists(f'{get_current_path()}/{new_folder}/valid'):
    os.mkdir(f'{get_current_path()}/{new_folder}/valid')
    
if not os.path.exists(f'{get_current_path()}/{new_folder}/test'):
    os.mkdir(f'{get_current_path()}/{new_folder}/test')
    
def make_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path

train_image_path = make_path(f'{get_current_path()}/{new_folder}/train/image')
train_segment_path = make_path(f'{get_current_path()}/{new_folder}/train/segmentation')
valid_image_path = make_path(f'{get_current_path()}/{new_folder}/valid/image')
valid_segment_path = make_path(f'{get_current_path()}/{new_folder}/valid/segmentation')
test_image_path = make_path(f'{get_current_path()}/{new_folder}/test/image')
test_segment_path = make_path(f'{get_current_path()}/{new_folder}/test/segmentation')  

src_folder_name = '3d_zero_padded_images'

src_train_image_path = f'{src_folder_name}/training/images'
src_valid_image_path = f'{src_folder_name}/validation/images'
src_test_image_path = f'{src_folder_name}/testing/images' 

src_train_mask_path = f'{src_folder_name}/training/labels'
src_valid_mask_path = f'{src_folder_name}/validation/labels'
src_test_mask_path = f'{src_folder_name}/testing/labels'  

In [18]:
def perform_image_slicing(root_path, saving_path):
    count = 0
    depth_list = []
    for file_name in tqdm(sorted(os.listdir(root_path))):
        # print(os.path.join(root_path, file_name))
        if file_name.endswith('.nii.gz'):
            img_id = str(file_name.split('_')[1])
            img = nib.load(os.path.join(root_path, file_name)).get_fdata()
            # normalizing the image between 0 and 1
            img=(img-img.min())/(max((img.max()-img.min()),1e-3))
            depth = img.shape[1]
            depth_list.append(depth)
            for j in range(depth):
                # j = 255
                new_path=os.path.join(saving_path, '{:05d}.npy'.format(j+count))
                new_img = torch.tensor(img[:, j:j+1, :].astype(np.float32))
                # new_img = new_img.permute(2, 0, 1)
                new_img = np.array(new_img)
                # print(new_img.shape)
                new_img = np.squeeze(new_img, axis=1)
                # print(new_img.shape)
                new_img = rotate(new_img, 90)
                new_img = np.expand_dims(new_img, axis=0)
            #     plt.imshow(new_img[0], cmap='gray')
            #     plt.axis('off')
            #     plt.show()
            #     break
            # break
                np.save(new_path, new_img)
            count += depth
    return depth_list
train_depth_list = perform_image_slicing(root_path=src_train_image_path, saving_path=train_image_path)
valid_depth_list = perform_image_slicing(root_path=src_valid_image_path, saving_path=valid_image_path)
test_depth_list = perform_image_slicing(root_path=src_test_image_path, saving_path=test_image_path)



100%|██████████| 8/8 [01:42<00:00, 12.85s/it]


In [19]:
import json

depth_lists = {
    'train': train_depth_list,
    'valid': valid_depth_list,
    'test': test_depth_list
}

json_file_path = f'{new_folder}/depth_lists.json'

with open(json_file_path, 'w') as json_file:
    json.dump(depth_lists, json_file)


In [28]:
def perform_segment_slicing(root_path, saving_path):
    count = 0
    depth_list = []
    for file_name in tqdm(sorted(os.listdir(root_path))):
        # print(os.path.join(root_path, file_name))
        seg_id = str(file_name.split('_')[1]).split('.')[0]
        seg = nib.load(os.path.join(root_path, file_name)).get_fdata()
        seg_no_cancer=np.where(seg>0,1,0).astype(np.uint8)
        depth = seg_no_cancer.shape[1]
        depth_list.append(depth)
        for j in range(depth):
            # j=150
            new_path=os.path.join(saving_path, '{:05d}.npy'.format(j+count))
            seg_1ch=torch.tensor(seg_no_cancer[:, j:j+1, :],dtype=torch.int64)
            seg_1ch = np.array(seg_1ch)
            seg_1ch = np.squeeze(seg_1ch, axis=1)
            seg_1ch = rotate(seg_1ch, 90)
            seg_1ch = np.expand_dims(seg_1ch, axis=2)
            seg_1ch = torch.tensor(seg_1ch)
            # print(seg_1ch.shape)
            seg_2ch=F.one_hot(seg_1ch,num_classes=2)
            # print(seg_2ch.shape)            
            seg_2ch=torch.squeeze(seg_2ch.permute(3,0,1,2))
            seg_2ch=np.array(seg_2ch,dtype=np.uint8)
            # print(seg_2ch.shape)
            # plt.imshow(seg_2ch[1,:,:], cmap='gray')
            # plt.show()
            np.save(new_path,seg_2ch)
        #     break
        # break
        count += depth

perform_segment_slicing(root_path=src_train_mask_path, saving_path=train_segment_path)
perform_segment_slicing(root_path=src_valid_mask_path, saving_path=valid_segment_path)
perform_segment_slicing(root_path=src_test_mask_path, saving_path=test_segment_path)

100%|██████████| 8/8 [02:05<00:00, 15.70s/it]
